In [217]:
"""Opportunistic Graph Search"""

import numpy as np
import random
import math

from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

In [218]:
def random_node(r=10, center=(0,0)):
    alpha = 2 * math.pi * random.random()
    
    # random radius
    r = r * math.sqrt(random.random())
    
    # calculating coordinates
    x = r * math.cos(alpha) + center[0]
    y = r * math.sin(alpha) + center[1]
    
    return (x, y)

In [241]:
def init_paths(num_riders, hub_coord):
    riders = ['r{}'.format(x) for x in range(num_riders)]
    paths = {}
    for rider in riders:
        paths[rider] = {
            'nodes_coord': [hub_coord],
            'time_tracker': [],
            'path': [],
            'must_leave': float('inf'),
            'will_return': float('inf'),
            'active': False,
            'completed': []
        }
        
    return paths

In [220]:
def compute_time_matrix(nodes):
    tm = [[0 for x in range(len(nodes))] for y in range(len(nodes))]
    
    for row, start in enumerate(nodes):
        for col, end in enumerate(nodes):
            x = np.array(start)
            y = np.array(end)
            distance = round(np.linalg.norm(x - y)) # Round to nearest integer
            
            if (end == (0,0)) | (row == col):
                tm[row][col] = distance 
            else:
                tm[row][col] = distance + 5 # We account for the handling time in the time to execute the path
       
    return tm

In [221]:
def nodes_match(nodes, new_nodes):
    for idx, _ in enumerate(nodes):
        if idx not in new_nodes:
            return False
    
    return True

In [222]:
def describe_path(rider, path):
    print('Rider {}'.format(rider))
    print('Status {}'.format(path['active']))
    
    all_path = 'Hub => '
    for node in path['path'][1:]:
        all_path += 'Node {} => '.format(node)
    all_path += 'Hub'
    
    print('Time on path: {}'.format(path['will_return'] - path['must_leave']))

In [223]:
def tsp_path(time_matrix):
    manager = pywrapcp.RoutingIndexManager(len(time_matrix), 1, 0) # Time_matrix, num_riders, depot
    routing = pywrapcp.RoutingModel(manager)
    
    # Time callback, gets the time it takes to travel between two nodes.
    def time_callback(from_index, to_index):
        # Convert from routing variable Index to time matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return time_matrix[from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(time_callback)
    
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # Search parameters.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)
    
    # Solution.
    solution = routing.SolveWithParameters(search_parameters)
    
    return manager, routing, solution

In [224]:
def get_routes(solution, routing, manager):
  routes = []
  for route_nbr in range(routing.vehicles()):
    index = routing.Start(route_nbr)
    route = [manager.IndexToNode(index)]
    while not routing.IsEnd(index):
      index = solution.Value(routing.NextVar(index))
      route.append(manager.IndexToNode(index))
    routes.append(route)
  return routes

In [271]:
def assign_order(order_coord, inputs, paths):
    print("New order coord", order_coord)
    # Try assign to available riders.
    order_assigned = False
    for rider, path in paths.items():
        print('RIDER {}'.format(rider), ' is active {}'.format(path['active']))
        if path['active']: continue # Cannot assign order to a rider already employed on a path.
        
        # Create the data to optimize the path.
        time_tracker = path['time_tracker'].copy()
        time_tracker.append(inputs['time'])
        nodes_coord = path['nodes_coord'].copy() 
        nodes_coord.append(order_coord) # Add the node to the established path of the rider.
        time_matrix = compute_time_matrix(nodes_coord) # Update the time matrix with the new nodes.
        
        print('Nodes coordinates', nodes_coord)
        
        manager, routing, solution = tsp_path(time_matrix)
        routes = get_routes(solution, routing, manager)
        
        new_nodes = routes[0][1:-1] # Remove first and last.
        
        # Check if path is within conditions. 
        total_time = 0
        last_node = 0
        must_leave = float('inf')
        for node in new_nodes:   
            total_time += time_matrix[last_node][node]
            last_node = node
            
            # Check if time permits
            time_permits = time_tracker[node - 1] + 30 > inputs['time'] + total_time # Time to get there
            if not time_permits: break
            if time_tracker[node - 1] + 30 - total_time < must_leave:
                must_leave = time_tracker[node - 1] + 30 - total_time
        
        total_time += time_matrix[last_node][0]
        
        # Add the path.
        paths[rider]['nodes_coord'] = nodes_coord
        paths[rider]['time_tracker'] = time_tracker
        paths[rider]['path'] = routes[0]
        paths[rider]['must_leave'] = must_leave
        paths[rider]['will_return'] = inputs['time'] + total_time
        paths[rider]['active'] = must_leave == inputs['time']          

        # Order has been assigned
        order_assigned = True
        break
    
    # Handle edge cases.
    if not order_assigned:
        filtered_paths = {k: v for k, v in paths.items() if not v['active']}
        print('FILTERED',len(filtered_paths))
        # Closest rider
        if len(filtered_paths) > 0:
            rider = ''
            new_path = []
            for rider, path in paths.items():
                time_tracker = path['time_tracker'].copy()
                time_tracker.append(inputs['time'])
                nodes_coord = path['nodes_coord'].copy() 
                nodes_coord.append(order_coord) # Add the node to the established path of the rider.
                time_matrix = compute_time_matrix(nodes_coord) # Update the time matrix with the new nodes.
                
                
                
        
        # Assign to rider on road
        else:
        
        
        
        
    return paths

In [272]:
def describe_paths(paths):
    total_paths = 0
    total_orders = 0
    for rider, path in paths.items():
        print('Rider {}'.format(rider))
        
        orders_handled = 0
        paths_handled = len(path['completed'])
        for path in path['completed']:
            orders_handled += len(path[1:-1])
            
        print('  > Paths handled: {}'.format(paths_handled))
        print('  > Orders handled: {}\n'.format(orders_handled))
        
        total_paths += paths_handled
        total_orders += orders_handled
    
    print('Total paths: {}'.format(total_paths))
    print('Total orders: {}'.format(total_orders))

In [273]:
# Inputs.
num_orders = 36
timeframe = 180 # minutes
num_riders = 2

cooking_time = 5
delivery_max = 30

inputs = {
    'time': 0,
    'hub_coord': (0,0),
    'prep_time': 5,
    'delivery_max': 30,
    'verbose': True,
    'open_orders': []
}

# ---------------------------
orders = [random.randrange(timeframe) for x in range(num_orders)] # Can be improved with density function
orders.sort()
# ---------------------------

paths = init_paths(num_riders, hub_coord)
while timeframe + 30 >= inputs['time']:
    print('time', inputs['time'])
    # Update riders active status.
    # - False, waiting to leave
    # - True, active on a path
    for rider, path in paths.items():
        # Rider becomes active.
        if path['must_leave'] == inputs['time']:
            paths[rider]['active'] = True
            
        # Rider becomes inactive, reset all paths specifics.
        if path['will_return'] == inputs['time']:
            paths[rider]['completed'].append(path['path'])
            paths[rider]['nodes_coord'] = [(0,0)]
            paths[rider]['time_tracker'] = []
            paths[rider]['must_leave'] = float('inf')
            paths[rider]['will_return'] = float('inf')
            paths[rider]['active'] = False
            print('PATH', path['path'])
        
        # Describe the riders path.
        if inputs['verbose'] & path['active']: describe_path(rider, path)
        
    
    for order in orders:
        if order == inputs['time']:
            print('New order')
            order_coord = random_node()
            print(order_coord)
            # Assign order to rider.
            new_paths = assign_order(order_coord, inputs, paths)
            paths = new_paths
    
    # Update time passaed.
    inputs['time'] += 1
    

time 0
time 1
time 2
time 3
time 4
time 5
time 6
time 7
time 8
time 9
time 10
New order
(0.15993635639080384, 9.263179356429667)
New order coord (0.15993635639080384, 9.263179356429667)
RIDER r0  is active False
Nodes coordinates [(0, 0), (0.15993635639080384, 9.263179356429667)]
time 11
time 12
New order
(4.521271605779454, -7.692286715805952)
New order coord (4.521271605779454, -7.692286715805952)
RIDER r0  is active False
Nodes coordinates [(0, 0), (0.15993635639080384, 9.263179356429667), (4.521271605779454, -7.692286715805952)]
time 13
time 14
time 15
time 16
New order
(0.2895248063742069, -7.476649464266188)
New order coord (0.2895248063742069, -7.476649464266188)
RIDER r0  is active False
Nodes coordinates [(0, 0), (0.15993635639080384, 9.263179356429667), (4.521271605779454, -7.692286715805952), (0.2895248063742069, -7.476649464266188)]
time 17
time 18
time 19
time 20
time 21
Rider r0
Status True
Time on path: 48
time 22
Rider r0
Status True
Time on path: 48
time 23
Rider r0
St

In [267]:
describe_paths(paths)

Rider r0
  > Paths handled: 3
  > Orders handled: 15

Rider r1
  > Paths handled: 3
  > Orders handled: 10

Total paths: 6
Total orders: 25


In [278]:
paths['r0']['path']

[0, 3, 2, 1, 0]